# AMEX Jupyter Notebook

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

pd.options.display.max_rows = 200
pd.options.display.max_info_columns = 200


## Dataset Import

In [2]:
file = './dataset/train_data.csv'
df = pd.read_csv(file)

## Dataset Inpection

In [3]:
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [5]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Data columns (total 190 columns):
 #    Column       Non-Null Count    Dtype  
---   ------       --------------    -----  
 0    customer_ID  5531451 non-null  object 
 1    S_2          5531451 non-null  object 
 2    P_2          5485466 non-null  float64
 3    D_39         5531451 non-null  float64
 4    B_1          5531451 non-null  float64
 5    B_2          5529435 non-null  float64
 6    R_1          5531451 non-null  float64
 7    S_3          4510907 non-null  float64
 8    D_41         5529435 non-null  float64
 9    B_3          5529435 non-null  float64
 10   D_42         791314 non-null   float64
 11   D_43         3873055 non-null  float64
 12   D_44         5257132 non-null  float64
 13   B_4          5531451 non-null  float64
 14   D_45         5529434 non-null  float64
 15   B_5          5531451 non-null  float64
 16   R_2          5531451 non-null  float64
 17   D_46         4319752 non-

In [6]:
N, D = df.shape

In [7]:
null_counts = df.isnull().sum()

In [12]:
null_percentages = null_counts / N * 100
has_NaN = null_percentages[null_percentages > 0]
print("No. of features with NaN values: ", has_NaN.size)
print(f"The feature with the most NaN is {has_NaN.idxmax()} ({has_NaN.max(): .2f}%)")
print(f"The feature with the most NaN is {has_NaN.idxmin()} ({has_NaN.min(): .10f}%)")

No. of features with NaN values:  122
The feature with the most NaN is D_87 ( 99.93%)
The feature with the most NaN is R_7 ( 0.0000180784%)


### Observation:
- __122 / 190__ features has NaN values. In some features, the percentage of NaN values could go up to 99.9%. 
- An initial preprocessing approach would be to remove these features such high number of NaN values. However, we have around 5 million data points, so 0.01% of the data is equal to approximately 500 data points. _Should we remove these 500 data points?_
- We should consider using some clustering algorithm to predict the missing values for these data points.
- Run the following cells to see a more detailed summary of NaN values in the dataset. _Should we consider doing a mixture of preprocessing where each feature has a different processing method?_

In [13]:
NaN_summary = pd.DataFrame()
NaN_summary["Features"] = df.columns
NaN_summary["No. NaN"] = null_counts.values
NaN_summary["No. Data"] = [N] * D
NaN_summary["NaN Percent"] = null_percentages.values

In [14]:
NaN_summary.loc[NaN_summary["NaN Percent"] > 0].sort_values(by="NaN Percent")

,Features,No. NaN,No. Data,NaN Percent
80,R_7,1,5531451,0.000018
100,R_14,1,5531451,0.000018
152,B_40,53,5531451,0.000958
142,B_37,56,5531451,0.001012
96,R_12,56,5531451,0.001012
117,R_20,75,5531451,0.001356
21,B_6,233,5531451,0.004212
131,S_23,445,5531451,0.008045
134,S_26,634,5531451,0.011462
171,B_41,690,5531451,0.012474


## Convert Pandas DataFrame to Numpy matrix

### Non-numerical features
- Column idx = 0:
    - Name: customer_ID
    - Data type: string
    - customer id
- Column idx = 2:
    - Name: S_2
    - Data type: string
    - datatime
- Column idx = 53:
    - Name: D_63
    - Data type: string
    - Has NaN: false
    - Unique Values: ['CR', 'CO', 'CL', 'XZ', 'XM', 'XL']
- Column idx = 54:
    - Name: D_64
    - Data type: string
    - Has NaN: true
    - Unique Values: ['O', 'R', nan, 'U', '-1']

In [4]:
original_data = df.to_numpy(copy=True, na_value = np.nan)

In [4]:
df.iloc[:,53].unique()

array(['CR', 'CO', 'CL', 'XZ', 'XM', 'XL'], dtype=object)

In [8]:
df.iloc[:,53].isnull().sum()

0

In [5]:
df.columns[53]

'D_63'

In [6]:
df.iloc[:,54].unique()

array(['O', 'R', nan, 'U', '-1'], dtype=object)

In [7]:
df.columns[54]

'D_64'

In [5]:
N, D = original_data.shape
numerical_data = np.empty((N,0), dtype=np.float64)
# concatenate feature idx = 2 to feature idx = 52 (inclusive)
numerical_data = np.concatenate((numerical_data, original_data[:, 2: 53].astype(np.float64)), axis=1)
# concatenate feature idx = 55 to the last feature
numerical_data = np.concatenate((numerical_data, original_data[:, 55:].astype(np.float64)), axis=1)

#### Replace NaN value by the mean in numerical features

In [6]:
is_nan = np.isnan(numerical_data)
is_real = np.invert(is_nan)
zero_filled = np.where(is_nan, 0, numerical_data)
no_real_values = np.sum(is_real, axis=0)
mean_features = np.sum(zero_filled, axis=0, keepdims=True) / no_real_values[np.newaxis,:]
mean_filled_data = np.where(is_nan, mean_features, numerical_data)

#### Replace NaN value in column idx = 54 (name = D_64) with the mode

In [7]:
D_64_mode = df["D_64"].mode()[0]
D_64_mode

'O'

In [8]:
D_64_data = original_data[:, 54].astype(str)
D_64_data

array(['O', 'O', 'O', ..., 'O', 'O', 'O'], dtype='<U3')

In [19]:
np.unique(D_64_data)

array(['-1', 'O', 'R', 'U', 'nan'], dtype='<U3')

In [9]:
D_64_mode_filled_data = np.where(D_64_data == 'nan', D_64_mode, D_64_data)
D_64_mode_filled_data

array(['O', 'O', 'O', ..., 'O', 'O', 'O'], dtype='<U3')

#### Combine all data into 1 matrix, bot numerical and string

In [10]:
part1 = original_data[:, 0:2]
part2 = mean_filled_data[:,:51]
part3 = original_data[:, 53][:, np.newaxis]
part4 = D_64_mode_filled_data[:, np.newaxis]
part5 =  mean_filled_data[:,51:]

In [ ]:
combined_data = np.concatenate((part1, part2, part3, part4, part5), axis=1)

#### Create a pandas dataframe

In [ ]:
index_values = np.array([i for i in range(N)], dtype=int)
column_values = df.columns
preprocessing2 = pd.DataFrame(data = combined_data, index=index_values, columns=column_values)

## Preprocessing data

### 1. Filling missing values with 0

In [15]:
preprocessing_1 = df.fillna(0)

#### Export to a csv file

In [20]:
filepath = Path("DataCleaning/csv/fill0.csv")
filepath.parent.mkdir(parents=True, exist_ok=True)
preprocessing_1.to_csv(filepath)

### 2. Filling missing values with mean/median

The train_data.csv has 190 features: 4 are string (`dtype=object`) while the rest are numerical (`dtype=float64`).
- Numerical features: NaN values are replaced with the mean.
- String features: NaN values are replaced with the mode.

In [ ]:
# create a data frame that map each column name to a replacement values
N, D = df.shape
# create a placeholder array to hold the replacement values
replacement_values = np.empty((D,), dtype=object)
# The replacement values for column 0, 1, and 53 are "" because these columns do not have NaN values
replacement_values[0], replacement_values[1], replacement_values[53] = "", "", ""
# The replacement values for column 54 is the mode of the feature
replacement_values[54] = df.iloc[:,54].mode()[0]
# the replacement values for the remaining columns is the mean
replacement_values[2:53] = df.iloc[:, 2: 53].mean(axis=0).values
replacement_values[55:] = df.iloc[:, 55:].mean(axis=0).values
# checking
replacement_values

In [4]:
replacements = {}
for column_index, column_name in enumerate(df.columns):
    replacements[column_name] = replacement_values[column_index]

In [5]:
preprocessing_2 = df.fillna(value=replacements)

#### Checking correctness

In [13]:
for column_name in df.columns:
    correctness = preprocessing_2[column_name].isnull().sum()
    if correctness != 0:
        print(f"Feature {column_name} has all NaN values replaced with the mean/mode: ", False)
        break

#### Export to a csv file

In [14]:
def export_to_csv(dataframe: pd.DataFrame, filename: str):
    filepath = Path(f"DataCleaning/csv/{filename}.csv")
    filepath.parent.mkdir(parents=True, exist_ok=True)
    dataframe.to_csv(filepath)

In [15]:
export_to_csv(preprocessing_2, "fillmeanmode")

## Download Preprocessed Data

Preprocessed data is stored on an Azure General-Purpose Storage Account. Use the following code to download the data into your local machine.

In [1]:
from DataRetrieval.download import download_preprocessed_data

# download preprocessed data where NaN values are replaced with mean (for numerical features) and mode (for string features)
download_preprocessed_data(mode="mean")

Downloaded 0 GB of data.  0.01% completed.
Downloaded 1 GB of data.  5.46% completed.
Downloaded 2 GB of data.  10.92% completed.
Downloaded 3 GB of data.  16.38% completed.
Downloaded 4 GB of data.  21.84% completed.
Downloaded 5 GB of data.  27.30% completed.
Downloaded 6 GB of data.  32.76% completed.
Downloaded 7 GB of data.  38.22% completed.
Downloaded 8 GB of data.  43.68% completed.
Downloaded 9 GB of data.  49.14% completed.
Downloaded 10 GB of data.  54.59% completed.
Downloaded 11 GB of data.  60.05% completed.
Downloaded 12 GB of data.  65.51% completed.
Downloaded 13 GB of data.  70.97% completed.
Downloaded 14 GB of data.  76.43% completed.
Downloaded 15 GB of data.  81.89% completed.
Downloaded 16 GB of data.  87.35% completed.
Downloaded 17 GB of data.  92.81% completed.
Downloaded 18 GB of data.  98.27% completed.


## Note

### Project Proposal Meeting
1. Data Understanding
    - there are 190 columns => 188 features (since the first column is index and the 2nd column is customer ID) 
    - 5,531,451 data points
    - 1 feature is the date. Could we elimiate it?
    - some feature has "NaN" value. How do we represent it? 
        run regression to predict it (statiscal analysis: modal, meadian, put the data on a distribution and see where the missing data would lie)
        0
        remove the columns
    
    => find a paper to read about how to deal with missing values

2. Papers:
    https://www.kaggle.com/competitions/amex-default-prediction/discussion/327135   

3. Team meeting for proposal recording on Thursday 10/6 at 5:30 PM

4. Trello Board: https://trello.com/b/F0tEjnUX/ml-project

5. Gantt Chart: https://docs.google.com/spreadsheets/d/1NwSPawBI_k9x3xHloXmnbROMbCaqwuFalB0XVgNrCJ8/edit?usp=sharing